# Code for segmenting nuclei

In [ ]:
import tifffile
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm, trange
import pandas as pd
import nd2
import glob
from cellpose import models, denoise
from scipy import ndimage
import pyclesperanto_prototype as cle
from multiprocessing import Pool
from itertools import product
from skimage.measure import regionprops
from skimage.util import view_as_windows
import seaborn as sns

In [ ]:

well_exclude = {1 : [],
                2: ['D11', 'E03'],  # contaminated
                3 : ['D12', 'E12', 'F01', 'H01', 'H05',
                     'A05', 'C01', 'E07', 'H08', 'H11'],
                4 : ['A12', 'C10', 'D04', 'D12', 'E08'],
                5 : ['C09', 'E05', 'E11', 'G03', 'G07'],
                6 : ['B08', 'C03', 'D05', 'F11', 'H02'],
                7 : ['B10'],
                8 : ['A04', 'A08', 'A11', 'B05', 'B06', 'B08', 'C09',
                    'D10', 'E11', 'F03', 'G08'],
                9 : ['A12'],
                10 : ['A08', 'A10', 'A11', 'B09', 'E03', 'E10', 'G08'],
                11 : [],
                12 : [],
                13 : ['B05'],
                14 : ['B08'],
                15 : [],
                'H1' : [],
                'H2' : []}


# points are zero indexed
point_exclude = {
    1 : {
        're_manual' : ['P51_5_A11_001_9', 'P51_5_A11_001_10', 'P51_5_A11_001_11',
                       'P47_3_D01_001_13', 'P51_4_A03_001_10', 'P51_4_A03_001_11']
    }
}


def get_info(plate_id, manual=False, re_manual=False, label_index=False):
  if manual:
    raw_dir = f'/data/hschlueter/pbmc5/manual/Plate{plate_id}/'
  elif re_manual:
    raw_dir = f'/data/hschlueter/pbmc5/re_manual/Plate{plate_id}/'
  else:
    raw_dir = f'/data/hschlueter/pbmc5/raw/Plate{plate_id}/'
  files = glob.glob(f'{raw_dir}/*.nd2')

  if plate_id == 'H1':
    info_path = f'/data/hschlueter/pbmc5/layout/plate_17_layout.csv'
  elif plate_id == 'H2':
    info_path = f'/data/hschlueter/pbmc5/layout/plate_18_layout.csv'
  else:
    info_path = f'/data/hschlueter/pbmc5/layout/plate_{plate_id}_layout.csv'
  info = pd.read_csv(info_path, index_col=0)
  if plate_id == 1:
    # drop correction control for now as they didn't use them
    info = info[info['condition'] != 'Correction control'].copy()
    info = info.replace({'H1':'H01', 'H2':'H02', 'H3':'H03', 'H4':'H04', 'H5':'H05'})
  if plate_id == 2:
    healthy_replace = {'H1':'H04', 'PHC1':'H05', 'PHC2':'H06', 'PHC3':'H07'}
    info = info.replace(healthy_replace).replace({'Correction control' : 'Healthy'})
  if plate_id == 9:
    info = info.replace({'P33_4.1' : 'P33_3', 'P33_4.2' : 'P33_4'})

  if plate_id in ['H1', 'H2']:
    info['patient'] = info['sample']
  else:
    info[['patient', 'time']] = info['sample'].str.split('_', expand=True)
  info.loc[info['condition'] == 'Healthy', 'time'] = 0
  info = info[~info['time'].isna()]
  if label_index:
    info = info.reset_index()
    info['label'] = info['well']
    info = info.set_index('label')
  info['fname'] = ''
  for fname in files:
    if manual or re_manual:
      if 'neu' in fname:
        well = fname.split('_')[-3]
      else:
        if 'Well' in fname:
          well = fname.split('Well')[1][:3]
        else:
          well = fname.split('_')[-2]
    else:
      well = fname.split('Well')[1][:3]

    fname = fname.split('/')[-1].split('.nd2')[0]
    if well in info.index:
      info.loc[well, 'fname'] = fname
      if label_index:
        info.rename(index={well : fname}, inplace=True)
    elif well not in well_exclude[plate_id]:
      assert label_index
      info.loc[fname] = info[info['well'] == well].iloc[0]
      info.loc[fname, 'fname'] = fname

  # drop empty wells that were imaged anyway and empty wells for single replicates
  if label_index:
    info = info[~info['well'].isin(well_exclude[plate_id])]
  else:
    info = info[~info.index.isin(well_exclude[plate_id])]
  if manual or re_manual:
    info = info[info['fname'] != '']
  info['time'] = info['time'].astype(int)
  return info

In [ ]:
def get_normalized_3d_frame(info, label, i):
  # frame, z, c, x, y
  stacks = nd2.imread(info.loc[label, 'nd2'])
  img = stacks[i]
  # min-max scaled to [0,1]
  img -= img.min()
  return img / img.max()

In [ ]:
# caution: uses global info and save_dir


def segment_vornoi_otsu(args):
    label, i = args
    img_3d = get_normalized_3d_frame(info, label, i)
    if len(img_3d.shape) == 4:
      img_3d = img_3d[:, 0]  # DAPI
    seg = naive_segment_3d(img_3d)

    fname = info.loc[label, 'nd2'].split('/')[-1].split('.nd2')[0]
    tifffile.imwrite(
                save_dir + 'dna_projections_2d/' + fname + f'_{i}.tiff',
                img_3d.max(axis=0)
            )
    tifffile.imwrite(
                save_dir + 'labeled_segmentation_3d/' + fname + f'_{i}.tiff',
                seg,
            )


In [ ]:
def plot_stack(info, label, n):
  fig, axs = plt.subplots(1, n, figsize=(4*n, 4))
  for i in range(n):
    img_3d = get_normalized_3d_frame(info, label, i)
    if len(img_3d.shape) == 4:
      img_3d = img_3d[:, 0]  # DAPI

    axs[i].imshow(img_3d.max(axis=0))
  fig.tight_layout()

# Plate 1

In [ ]:
plate = 'Plate1'
s_plate = 'plate_1'

In [ ]:
raw_dir = f'/data/hschlueter/pbmc5/raw/{plate}/'
files = glob.glob(f'{raw_dir}/*.nd2')
layout_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_2d_layout.csv'
layout = pd.read_csv(layout_path, index_col=0)
layout

,01,02,03,04,05,06,07,08,09,10,11,12
A,PHC2,H1,P51_4,H4,P57_3,P132_3,P57_1,P51_3,P132_4,P46_3,P51_5,P57_4
B,P51_1,P57_1,P47_2,P132_1,PHC2,P108_3,P132_5,P47_4,P47_1,P51_2,P57_4,P68_4
C,P132_5,P57_5,H2,P46_5,P68_2,P47_5,P47_1,P150_4,P46_5,P46_3,PHC1,P150_1
D,P47_3,P132_4,H5,P68_1,P46_4,P68_3,P150_3,P51_5,P108_3,H2,P132_3,H5
E,P132_2,P108_2,P150_2,P150_2,H4,P132_1,P57_3,P51_1,P47_3,P108_4,P47_4,P150_5
F,P68_3,P46_1,P57_2,P46_1,P51_3,P57_2,P150_1,P68_1,P108_2,P51_2,P150_4,H3
G,P108_5,P108_1,P51_4,P57_5,P68_4,P150_5,P68_5,P46_2,P47_2,P108_4,P68_5,P46_2
H,P46_4,P150_3,P47_5,PHC1,P132_2,H3,P108_5,PHC3,PHC3,P68_2,H1,P108_1


In [ ]:
info_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_layout.csv'
info = pd.read_csv(info_path, index_col=0)
# drop correction control for now as they didn't use them
info = info[info['condition'] != 'Correction control'].copy()
info[['patient', 'time']] = info['sample'].str.split('_', expand=True)
info['nd2'] = ''
for fname in files:
  well = fname.split('Well')[1][:3]
  info.loc[well, 'nd2'] = fname

assert (info['nd2'] != '').all()
info

,condition,sample,patient,time,nd2
well,,,,,
A02,Healthy,H1,H1,None,/data/hschlueter/pbmc5/raw/Plate1/WellA02_Chan...
A03,Glioblastoma,P51_4,P51,4,/data/hschlueter/pbmc5/raw/Plate1/WellA03_Chan...
A04,Healthy,H4,H4,None,/data/hschlueter/pbmc5/raw/Plate1/WellA04_Chan...
A05,Glioblastoma,P57_3,P57,3,/data/hschlueter/pbmc5/raw/Plate1/WellA05_Chan...
A06,Glioblastoma,P132_3,P132,3,/data/hschlueter/pbmc5/raw/Plate1/WellA06_Chan...
...,...,...,...,...,...
H06,Healthy,H3,H3,None,/data/hschlueter/pbmc5/raw/Plate1/WellH06_Chan...
H07,Glioblastoma,P108_5,P108,5,/data/hschlueter/pbmc5/raw/Plate1/WellH07_Chan...
H10,Glioblastoma,P68_2,P68,2,/data/hschlueter/pbmc5/raw/Plate1/WellH10_Chan...


In [ ]:
save_dir = f'/home/hschlueter/data/pbmc5/seg_voronoi_otsu/{plate}/segmentation/'
options = list(product(info.index, np.arange(8)))
for args in tqdm(options):
  segment_vornoi_otsu(args)

  0%|          | 0/720 [00:00<?, ?it/s]

size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
s

# Plate 2

In [ ]:
plate = 'Plate2'
s_plate = 'plate_2'

# as we made these before deciding overlap
healthy_replace = {'H1':'H4', 'PHC1':'H5', 'PHC2':'H6', 'PHC3':'H7'}

In [ ]:
raw_dir = f'/data/hschlueter/pbmc5/raw/{plate}/'
files = glob.glob(f'{raw_dir}/*.nd2')
layout_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_2d_layout.csv'
layout = pd.read_csv(layout_path, index_col=0).replace(healthy_replace)
layout

,01,02,03,04,05,06,07,08,09,10,11,12
A,P52_3,P116_4,P116_3,H5,P15_2,P116_1,P124_2,P52_3,P15_3,P22_1,P15_2,P29_4
B,P15_4,P116_3,P16_5,P29_5,P104_1,P29_3,P116_2,P124_2,P29_3,H4,P124_1,P104_3
C,P104_4,P104_4,P29_1,P84_1,P124_3,P52_2,P124_4,P124_5,P116_5,P104_5,P15_1,P124_3
D,P84_4,P23_4,H4,P84_4,P23_5,H6,P84_5,P84_2,P22_2,P16_3,P84_3,P52_2
E,P84_2,H7,P84_3,P104_5,P16_3,P52_4,P52_5,P29_2,P15_5,P104_1,P52_1,P104_3
F,P23_4,P15_1,P104_2,P116_2,H6,P29_4,P22_1,P124_4,P124_5,P29_1,P29_5,P23_3
G,P15_4,P23_5,P52_4,P84_5,P16_2,P15_5,H7,P116_4,P29_2,P116_1,P16_1,H5
H,P16_5,P52_5,P104_2,P16_2,P116_5,P52_1,P22_2,P124_1,P15_3,P84_1,P23_3,P16_1


In [ ]:
files

['/data/hschlueter/pbmc5/raw/Plate2/WellE02_ChannelDAPI_Seq0047.nd2',
 '/data/hschlueter/pbmc5/raw/Plate2/WellE11_ChannelDAPI_Seq0056.nd2',
 '/data/hschlueter/pbmc5/raw/Plate2/WellB05_ChannelDAPI_Seq0018.nd2',
 '/data/hschlueter/pbmc5/raw/Plate2/WellA06_ChannelDAPI_Seq0004.nd2',
 '/data/hschlueter/pbmc5/raw/Plate2/WellE08_ChannelDAPI_Seq0053.nd2',
 '/data/hschlueter/pbmc5/raw/Plate2/WellB09_ChannelDAPI_Seq0014.nd2',
 '/data/hschlueter/pbmc5/raw/Plate2/WellA05_ChannelDAPI_Seq0003.nd2',
 '/data/hschlueter/pbmc5/raw/Plate2/WellF10_ChannelDAPI_Seq0060.nd2',
 '/data/hschlueter/pbmc5/raw/Plate2/WellG05_ChannelDAPI_Seq0073.nd2',
 '/data/hschlueter/pbmc5/raw/Plate2/WellE09_ChannelDAPI_Seq0054.nd2',
 '/data/hschlueter/pbmc5/raw/Plate2/WellD05_ChannelDAPI_Seq0041.nd2',
 '/data/hschlueter/pbmc5/raw/Plate2/WellH05_ChannelDAPI_Seq0088.nd2',
 '/data/hschlueter/pbmc5/raw/Plate2/WellC04_ChannelDAPI_Seq0026.nd2',
 '/data/hschlueter/pbmc5/raw/Plate2/WellD08_ChannelDAPI_Seq0038.nd2',
 '/data/hschlueter/p

In [ ]:
info_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_layout.csv'
info = pd.read_csv(info_path, index_col=0).replace(healthy_replace)
info[['patient', 'time']] = info['sample'].str.split('_', expand=True)
info['nd2'] = ''
for fname in files:
  well = fname.split('Well')[1][:3]
  info.loc[well, 'nd2'] = fname

print(info[info['nd2'] == ''])  # empty wells
info = info[info['nd2'] != '']
info

         condition sample patient time nd2
well                                      
A01   Glioblastoma  P52_3     P52    3    
D10   Glioblastoma  P16_3     P16    3    
G01   Glioblastoma  P15_4     P15    4    


,condition,sample,patient,time,nd2
well,,,,,
A02,Glioblastoma,P116_4,P116,4,/data/hschlueter/pbmc5/raw/Plate2/WellA02_Chan...
A03,Glioblastoma,P116_3,P116,3,/data/hschlueter/pbmc5/raw/Plate2/WellA03_Chan...
A04,Correction control,H5,H5,None,/data/hschlueter/pbmc5/raw/Plate2/WellA04_Chan...
A05,Glioblastoma,P15_2,P15,2,/data/hschlueter/pbmc5/raw/Plate2/WellA05_Chan...
A06,Glioblastoma,P116_1,P116,1,/data/hschlueter/pbmc5/raw/Plate2/WellA06_Chan...
...,...,...,...,...,...
H08,Glioblastoma,P124_1,P124,1,/data/hschlueter/pbmc5/raw/Plate2/WellH08_Chan...
H09,Glioblastoma,P15_3,P15,3,/data/hschlueter/pbmc5/raw/Plate2/WellH09_Chan...
H10,Glioblastoma,P84_1,P84,1,/data/hschlueter/pbmc5/raw/Plate2/WellH10_Chan...


In [ ]:
save_dir = f'/home/hschlueter/data/pbmc5/seg_voronoi_otsu/{plate}/segmentation/'
options = list(product(info.index, np.arange(11)))
for args in tqdm(options):
  segment_vornoi_otsu(args)

  0%|          | 0/1023 [00:00<?, ?it/s]

size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
s

# Plate 3

In [ ]:
plate = 'Plate3'
s_plate = 'plate_3'

In [ ]:
raw_dir = f'/data/hschlueter/pbmc5/raw/{plate}/'
files = glob.glob(f'{raw_dir}/*.nd2')
layout_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_2d_layout.csv'
layout = pd.read_csv(layout_path, index_col=0)
layout

,01,02,03,04,05,06,07,08,09,10,11,12
A,NaN,P115_2,P72_3,P50_3,P100_1,P115_3,P67_1,P100_4,H06,P67_5,P77_1,P100_5
B,P77_4,P77_5,P67_3,P67_3,P56_1,P67_5,P50_4,P125_5,P56_2,P115_3,P100_3,P56_4
C,P100_2,P67_1,P50_2,P125_1,P72_3,P100_2,P125_5,P56_1,P125_3,P115_2,P125_1,P77_3
D,P72_1,P41_4,P41_3,P67_4,P77_5,P72_2,P77_2,P77_1,P41_2,P41_5,P72_1,NaN
E,P125_4,H06,P56_4,P50_3,P100_4,P125_4,P41_1,P100_1,P41_2,P50_4,P41_1,NaN
F,NaN,P77_3,P56_3,P72_2,P77_4,H30,P100_3,P56_5,P67_2,P56_3,P125_2,P50_1
G,P125_3,H23,P100_5,P56_5,P50_1,P72_5,H30,H07,P67_2,P115_1,P125_2,P67_4
H,NaN,H07,P41_5,P77_2,NaN,P41_4,H23,P72_5,P50_2,P56_2,P115_1,P41_3


In [ ]:
files

['/data/hschlueter/pbmc5/raw/Plate3/WellB05_ChannelDAPI_Seq0018.nd2',
 '/data/hschlueter/pbmc5/raw/Plate3/WellD10_ChannelDAPI_Seq0037.nd2',
 '/data/hschlueter/pbmc5/raw/Plate3/WellD09_ChannelDAPI_Seq0038.nd2',
 '/data/hschlueter/pbmc5/raw/Plate3/WellA06_ChannelDAPI_Seq0004.nd2',
 '/data/hschlueter/pbmc5/raw/Plate3/WellG06_ChannelDAPI_Seq0076.nd2',
 '/data/hschlueter/pbmc5/raw/Plate3/WellE09_ChannelDAPI_Seq0055.nd2',
 '/data/hschlueter/pbmc5/raw/Plate3/WellB09_ChannelDAPI_Seq0014.nd2',
 '/data/hschlueter/pbmc5/raw/Plate3/WellH04_ChannelDAPI_Seq0091.nd2',
 '/data/hschlueter/pbmc5/raw/Plate3/WellA05_ChannelDAPI_Seq0003.nd2',
 '/data/hschlueter/pbmc5/raw/Plate3/WellH12_ChannelDAPI_Seq0083.nd2',
 '/data/hschlueter/pbmc5/raw/Plate3/WellH01_ChannelDAPI_Seq0094.nd2',
 '/data/hschlueter/pbmc5/raw/Plate3/WellE12_ChannelDAPI_Seq0058.nd2',
 '/data/hschlueter/pbmc5/raw/Plate3/WellC04_ChannelDAPI_Seq0026.nd2',
 '/data/hschlueter/pbmc5/raw/Plate3/WellG09_ChannelDAPI_Seq0079.nd2',
 '/data/hschlueter/p

In [ ]:
info_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_layout.csv'
info = pd.read_csv(info_path, index_col=0)
info[['patient', 'time']] = info['sample'].str.split('_', expand=True)
info['nd2'] = ''
for fname in files:
  well = fname.split('Well')[1][:3]
  info.loc[well, 'nd2'] = fname

print(info[(info['nd2'] == '') | info['patient'].isna()])  # empty wells
info = info[info['nd2'] != '']
info

     condition sample patient time  \
well                                 
A01        NaN    NaN     NaN  NaN   
D12        NaN    NaN     NaN  NaN   
E12        NaN    NaN     NaN  NaN   
F01        NaN    NaN     NaN  NaN   
H01        NaN    NaN     NaN  NaN   
H05        NaN    NaN     NaN  NaN   

                                                    nd2  
well                                                     
A01                                                      
D12   /data/hschlueter/pbmc5/raw/Plate3/WellD12_Chan...  
E12   /data/hschlueter/pbmc5/raw/Plate3/WellE12_Chan...  
F01   /data/hschlueter/pbmc5/raw/Plate3/WellF01_Chan...  
H01   /data/hschlueter/pbmc5/raw/Plate3/WellH01_Chan...  
H05   /data/hschlueter/pbmc5/raw/Plate3/WellH05_Chan...  


,condition,sample,patient,time,nd2
well,,,,,
A02,Head & Neck,P115_2,P115,2,/data/hschlueter/pbmc5/raw/Plate3/WellA02_Chan...
A03,Head & Neck,P72_3,P72,3,/data/hschlueter/pbmc5/raw/Plate3/WellA03_Chan...
A04,Head & Neck,P50_3,P50,3,/data/hschlueter/pbmc5/raw/Plate3/WellA04_Chan...
A05,Head & Neck,P100_1,P100,1,/data/hschlueter/pbmc5/raw/Plate3/WellA05_Chan...
A06,Head & Neck,P115_3,P115,3,/data/hschlueter/pbmc5/raw/Plate3/WellA06_Chan...
...,...,...,...,...,...
H08,Head & Neck,P72_5,P72,5,/data/hschlueter/pbmc5/raw/Plate3/WellH08_Chan...
H09,Head & Neck,P50_2,P50,2,/data/hschlueter/pbmc5/raw/Plate3/WellH09_Chan...
H10,Head & Neck,P56_2,P56,2,/data/hschlueter/pbmc5/raw/Plate3/WellH10_Chan...


In [ ]:
save_dir = f'/home/hschlueter/data/pbmc5/seg_voronoi_otsu/{plate}/segmentation/'
options = list(product(info.index, np.arange(11)))
for args in tqdm(options):
  segment_vornoi_otsu(args)

  0%|          | 0/1045 [00:00<?, ?it/s]

size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
s

# Plate 4

In [ ]:
plate = 'Plate4'
s_plate = 'plate_4'

In [ ]:
raw_dir = f'/data/hschlueter/pbmc5/raw/{plate}/'
files = glob.glob(f'{raw_dir}/*.nd2')
layout_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_2d_layout.csv'
layout = pd.read_csv(layout_path, index_col=0)
layout

,01,02,03,04,05,06,07,08,09,10,11,12
A,P129_5,P129_1,P137_1,P107_2,P114_1,P129_4,P149_5,P107_1,P129_2,P79_3,P137_3,NaN
B,P114_4,P120_2,P114_3,H40,P120_5,P79_4,P129_5,P145_1,P145_4,P120_4,P114_1,H40
C,P149_2,P130_1,P149_3,P137_1,P107_1,P129_1,H10,NaN,P130_3,P145_3,P79_5,P145_2
D,P149_4,P145_2,P149_5,P120_2,P120_5,P149_3,P129_3,P120_3,H30,P130_2,P149_2,P120_1
E,P79_1,P137_4,P107_2,H23,P137_3,P107_3,P130_5,P149_1,P114_2,P114_5,P149_4,P107_5
F,P114_2,P79_4,P107_4,P129_4,P114_4,P129_2,P130_2,P130_3,P79_5,P120_4,P137_2,P145_1
G,P107_3,P129_3,P145_5,P149_1,P79_3,P107_5,P130_4,H30,P120_1,P79_1,P130_4,P120_3
H,P137_4,H23,P145_3,P114_3,P107_4,P137_2,P130_1,P114_5,P130_5,H10,P145_4,P145_5


In [ ]:
files

['/data/hschlueter/pbmc5/raw/Plate4/WellG09_ChannelDAPI_Seq0075.nd2',
 '/data/hschlueter/pbmc5/raw/Plate4/WellB01_ChannelDAPI_Seq0023.nd2',
 '/data/hschlueter/pbmc5/raw/Plate4/WellH11_ChannelDAPI_Seq0080.nd2',
 '/data/hschlueter/pbmc5/raw/Plate4/WellH12_ChannelDAPI_Seq0079.nd2',
 '/data/hschlueter/pbmc5/raw/Plate4/WellE11_ChannelDAPI_Seq0053.nd2',
 '/data/hschlueter/pbmc5/raw/Plate4/WellF11_ChannelDAPI_Seq0056.nd2',
 '/data/hschlueter/pbmc5/raw/Plate4/WellG10_ChannelDAPI_Seq0076.nd2',
 '/data/hschlueter/pbmc5/raw/Plate4/WellF07_ChannelDAPI_Seq0060.nd2',
 '/data/hschlueter/pbmc5/raw/Plate4/WellE04_ChannelDAPI_Seq0047.nd2',
 '/data/hschlueter/pbmc5/raw/Plate4/WellA09_ChannelDAPI_Seq0008.nd2',
 '/data/hschlueter/pbmc5/raw/Plate4/WellC11_ChannelDAPI_Seq0032.nd2',
 '/data/hschlueter/pbmc5/raw/Plate4/WellE10_ChannelDAPI_Seq0052.nd2',
 '/data/hschlueter/pbmc5/raw/Plate4/WellH03_ChannelDAPI_Seq0088.nd2',
 '/data/hschlueter/pbmc5/raw/Plate4/WellH01_ChannelDAPI_Seq0090.nd2',
 '/data/hschlueter/p

In [ ]:
info_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_layout.csv'
info = pd.read_csv(info_path, index_col=0)
info[['patient', 'time']] = info['sample'].str.split('_', expand=True)
info['nd2'] = ''
for fname in files:
  well = fname.split('Well')[1][:3]
  info.loc[well, 'nd2'] = fname

print(info[(info['nd2'] == '') | info['patient'].isna()])  # empty wells
info = info[info['nd2'] != '']
info

        condition  sample patient time  \
well                                     
A12           NaN     NaN     NaN  NaN   
C08           NaN     NaN     NaN  NaN   
C10   Head & Neck  P145_3    P145    3   
D04   Head & Neck  P120_2    P120    2   
D12   Head & Neck  P120_1    P120    1   
E08   Head & Neck  P149_1    P149    1   

                                                    nd2  
well                                                     
A12   /data/hschlueter/pbmc5/raw/Plate4/WellA12_Chan...  
C08                                                      
C10                                                      
D04                                                      
D12                                                      
E08                                                      


,condition,sample,patient,time,nd2
well,,,,,
A01,Head & Neck,P129_5,P129,5,/data/hschlueter/pbmc5/raw/Plate4/WellA01_Chan...
A02,Head & Neck,P129_1,P129,1,/data/hschlueter/pbmc5/raw/Plate4/WellA02_Chan...
A03,Head & Neck,P137_1,P137,1,/data/hschlueter/pbmc5/raw/Plate4/WellA03_Chan...
A04,Head & Neck,P107_2,P107,2,/data/hschlueter/pbmc5/raw/Plate4/WellA04_Chan...
A05,Head & Neck,P114_1,P114,1,/data/hschlueter/pbmc5/raw/Plate4/WellA05_Chan...
...,...,...,...,...,...
H08,Head & Neck,P114_5,P114,5,/data/hschlueter/pbmc5/raw/Plate4/WellH08_Chan...
H09,Head & Neck,P130_5,P130,5,/data/hschlueter/pbmc5/raw/Plate4/WellH09_Chan...
H10,Healthy,H10,H10,None,/data/hschlueter/pbmc5/raw/Plate4/WellH10_Chan...


In [ ]:
save_dir = f'/home/hschlueter/data/pbmc5/seg_voronoi_otsu/{plate}/segmentation/'
options = list(product(info.index, np.arange(11)))
for args in tqdm(options):
  segment_vornoi_otsu(args)

  0%|          | 0/1001 [00:00<?, ?it/s]

size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
s

# Plate 5

In [ ]:
plate = 'Plate5'
s_plate = 'plate_5'

In [ ]:
raw_dir = f'/data/hschlueter/pbmc5/raw/{plate}/'
files = glob.glob(f'{raw_dir}/*.nd2')
layout_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_2d_layout.csv'
layout = pd.read_csv(layout_path, index_col=0)
layout

,01,02,03,04,05,06,07,08,09,10,11,12
A,P148_5,P78_2,H40,NaN,P63_5,P24_3,P18_5,P18_1,P135_1,P12_2,NaN,P135_4
B,P142_2,P148_4,P148_2,P18_1,P142_5,P142_1,P18_3,P135_2,P12_2,P63_4,P135_4,P161_4
C,P161_1,P135_2,P12_5,P142_3,P24_2,P142_1,P78_1,P18_5,P12_5,H31,P63_3,P63_1
D,P12_1,P24_1,P78_4,P148_3,P63_1,P24_3,P63_2,P12_4,H10,H31,P148_1,P78_5
E,P161_2,P12_3,P142_4,P63_2,P78_5,P135_5,H40,P135_3,P135_1,NaN,P12_3,P63_4
F,P12_4,P18_2,P161_1,H39,NaN,P78_2,P63_3,P12_1,P161_4,P142_2,P18_2,P78_4
G,H10,P24_1,P24_2,P161_3,P78_3,P63_5,P161_2,P148_1,H39,P148_5,P142_4,NaN
H,P135_5,P142_5,NaN,P148_2,P148_3,P78_1,P135_3,P18_3,P142_3,P161_3,P148_4,P78_3


In [ ]:
files

['/data/hschlueter/pbmc5/raw/Plate5/WellH10_ChannelDAPI_Seq0076.nd2',
 '/data/hschlueter/pbmc5/raw/Plate5/WellA06_ChannelDAPI_Seq0004.nd2',
 '/data/hschlueter/pbmc5/raw/Plate5/WellH06_ChannelDAPI_Seq0080.nd2',
 '/data/hschlueter/pbmc5/raw/Plate5/WellE08_ChannelDAPI_Seq0051.nd2',
 '/data/hschlueter/pbmc5/raw/Plate5/WellA05_ChannelDAPI_Seq0003.nd2',
 '/data/hschlueter/pbmc5/raw/Plate5/WellD02_ChannelDAPI_Seq0043.nd2',
 '/data/hschlueter/pbmc5/raw/Plate5/WellF11_ChannelDAPI_Seq0055.nd2',
 '/data/hschlueter/pbmc5/raw/Plate5/WellF07_ChannelDAPI_Seq0059.nd2',
 '/data/hschlueter/pbmc5/raw/Plate5/WellC03_ChannelDAPI_Seq0024.nd2',
 '/data/hschlueter/pbmc5/raw/Plate5/WellF10_ChannelDAPI_Seq0056.nd2',
 '/data/hschlueter/pbmc5/raw/Plate5/WellC01_ChannelDAPI_Seq0022.nd2',
 '/data/hschlueter/pbmc5/raw/Plate5/WellG09_ChannelDAPI_Seq0071.nd2',
 '/data/hschlueter/pbmc5/raw/Plate5/WellH05_ChannelDAPI_Seq0081.nd2',
 '/data/hschlueter/pbmc5/raw/Plate5/WellH04_ChannelDAPI_Seq0082.nd2',
 '/data/hschlueter/p

In [ ]:
info_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_layout.csv'
info = pd.read_csv(info_path, index_col=0)
info[['patient', 'time']] = info['sample'].str.split('_', expand=True)
info['nd2'] = ''
for fname in files:
  well = fname.split('Well')[1][:3]
  info.loc[well, 'nd2'] = fname

print(info[(info['nd2'] == '') | info['patient'].isna()])  # empty wells
info = info[info['nd2'] != '']
info

        condition  sample patient time nd2
well                                      
A04           NaN     NaN     NaN  NaN    
A11           NaN     NaN     NaN  NaN    
C09   Head & Neck   P12_5     P12    5    
E05   Head & Neck   P78_5     P78    5    
E10           NaN     NaN     NaN  NaN    
E11   Head & Neck   P12_3     P12    3    
F05           NaN     NaN     NaN  NaN    
G03   Head & Neck   P24_2     P24    2    
G07   Head & Neck  P161_2    P161    2    
G12           NaN     NaN     NaN  NaN    
H03           NaN     NaN     NaN  NaN    


,condition,sample,patient,time,nd2
well,,,,,
A01,Head & Neck,P148_5,P148,5,/data/hschlueter/pbmc5/raw/Plate5/WellA01_Chan...
A02,Head & Neck,P78_2,P78,2,/data/hschlueter/pbmc5/raw/Plate5/WellA02_Chan...
A03,Healthy,H40,H40,None,/data/hschlueter/pbmc5/raw/Plate5/WellA03_Chan...
A05,Head & Neck,P63_5,P63,5,/data/hschlueter/pbmc5/raw/Plate5/WellA05_Chan...
A06,Head & Neck,P24_3,P24,3,/data/hschlueter/pbmc5/raw/Plate5/WellA06_Chan...
...,...,...,...,...,...
H08,Head & Neck,P18_3,P18,3,/data/hschlueter/pbmc5/raw/Plate5/WellH08_Chan...
H09,Head & Neck,P142_3,P142,3,/data/hschlueter/pbmc5/raw/Plate5/WellH09_Chan...
H10,Head & Neck,P161_3,P161,3,/data/hschlueter/pbmc5/raw/Plate5/WellH10_Chan...


In [ ]:
save_dir = f'/home/hschlueter/data/pbmc5/seg_voronoi_otsu/{plate}/segmentation/'
options = list(product(info.index, np.arange(11)))
for args in tqdm(options):
  segment_vornoi_otsu(args)

  0%|          | 0/935 [00:00<?, ?it/s]

size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
s

# Plate 6

In [ ]:
plate = 'Plate6'
s_plate = 'plate_6'

In [ ]:
raw_dir = f'/data/hschlueter/pbmc5/raw/{plate}/'
files = glob.glob(f'{raw_dir}/*.nd2')
layout_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_2d_layout.csv'
layout = pd.read_csv(layout_path, index_col=0)
layout

,01,02,03,04,05,06,07,08,09,10,11,12
A,P94_2,P94_4,H31,P166_2,H22,P147_2,P14_4,P154_4,P14_3,P55_2,P166_4,P44_2
B,P55_1,P14_1,P55_4,P98_4,P154_2,P154_4,P94_4,P106_2,P110_4,P154_3,H37,P110_4
C,P106_3,H39,P106_2,P110_2,P166_4,P98_2,P154_3,P94_2,P98_3,P147_3,P147_1,P14_1
D,P55_4,P98_1,P94_1,P55_2,P94_3,H31,H39,P14_2,P44_2,P166_1,P106_1,H37
E,P44_3,P154_5,P110_3,P14_5,P106_3,P110_1,P55_5,P14_2,P166_5,P44_1,P110_3,P98_5
F,P94_1,P14_4,P94_3,P55_3,P98_5,P14_3,P166_1,P44_3,P147_2,P154_1,P98_2,P55_3
G,P154_1,P166_3,P94_5,P166_2,P55_5,H22,P98_3,P14_5,P55_1,P166_5,P110_5,P147_3
H,P94_5,P98_1,P110_5,P110_1,P154_5,P147_1,P44_1,P154_2,P166_3,P98_4,P110_2,P106_1


In [ ]:
files

['/data/hschlueter/pbmc5/raw/Plate6/WellG02_ChannelDAPI_Seq0069.nd2',
 '/data/hschlueter/pbmc5/raw/Plate6/WellB05_ChannelDAPI_Seq0018.nd2',
 '/data/hschlueter/pbmc5/raw/Plate6/WellD12_ChannelDAPI_Seq0034.nd2',
 '/data/hschlueter/pbmc5/raw/Plate6/WellE11_ChannelDAPI_Seq0055.nd2',
 '/data/hschlueter/pbmc5/raw/Plate6/WellH03_ChannelDAPI_Seq0089.nd2',
 '/data/hschlueter/pbmc5/raw/Plate6/WellH10_ChannelDAPI_Seq0082.nd2',
 '/data/hschlueter/pbmc5/raw/Plate6/WellG09_ChannelDAPI_Seq0076.nd2',
 '/data/hschlueter/pbmc5/raw/Plate6/WellA09_ChannelDAPI_Seq0008.nd2',
 '/data/hschlueter/pbmc5/raw/Plate6/WellC11_ChannelDAPI_Seq0032.nd2',
 '/data/hschlueter/pbmc5/raw/Plate6/WellD02_ChannelDAPI_Seq0043.nd2',
 '/data/hschlueter/pbmc5/raw/Plate6/WellE10_ChannelDAPI_Seq0054.nd2',
 '/data/hschlueter/pbmc5/raw/Plate6/WellG01_ChannelDAPI_Seq0068.nd2',
 '/data/hschlueter/pbmc5/raw/Plate6/WellH01_ChannelDAPI_Seq0090.nd2',
 '/data/hschlueter/pbmc5/raw/Plate6/WellH09_ChannelDAPI_Seq0083.nd2',
 '/data/hschlueter/p

In [ ]:
info_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_layout.csv'
info = pd.read_csv(info_path, index_col=0)
info[['patient', 'time']] = info['sample'].str.split('_', expand=True)
info['nd2'] = ''
for fname in files:
  well = fname.split('Well')[1][:3]
  info.loc[well, 'nd2'] = fname

print(info[(info['nd2'] == '') | info['patient'].isna()])  # empty wells
info = info[info['nd2'] != '']
info

        condition  sample patient time nd2
well                                      
B08   Head & Neck  P106_2    P106    2    
C03   Head & Neck  P106_2    P106    2    
D05   Head & Neck   P94_3     P94    3    
F11   Head & Neck   P98_2     P98    2    
H02   Head & Neck   P98_1     P98    1    


,condition,sample,patient,time,nd2
well,,,,,
A01,Head & Neck,P94_2,P94,2,/data/hschlueter/pbmc5/raw/Plate6/WellA01_Chan...
A02,Head & Neck,P94_4,P94,4,/data/hschlueter/pbmc5/raw/Plate6/WellA02_Chan...
A03,Healthy,H31,H31,None,/data/hschlueter/pbmc5/raw/Plate6/WellA03_Chan...
A04,Head & Neck,P166_2,P166,2,/data/hschlueter/pbmc5/raw/Plate6/WellA04_Chan...
A05,Healthy,H22,H22,None,/data/hschlueter/pbmc5/raw/Plate6/WellA05_Chan...
...,...,...,...,...,...
H08,Head & Neck,P154_2,P154,2,/data/hschlueter/pbmc5/raw/Plate6/WellH08_Chan...
H09,Head & Neck,P166_3,P166,3,/data/hschlueter/pbmc5/raw/Plate6/WellH09_Chan...
H10,Head & Neck,P98_4,P98,4,/data/hschlueter/pbmc5/raw/Plate6/WellH10_Chan...


In [ ]:
save_dir = f'/home/hschlueter/data/pbmc5/seg_voronoi_otsu/{plate}/segmentation/'
options = list(product(info.index, np.arange(11)))
for args in tqdm(options):
  segment_vornoi_otsu(args)

  0%|          | 0/1001 [00:00<?, ?it/s]

size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
s

# Plate 7

In [ ]:
plate = 'Plate7'
s_plate = 'plate_7'

In [ ]:
raw_dir = f'/data/hschlueter/pbmc5/raw/{plate}/'
files = glob.glob(f'{raw_dir}/*.nd2')
layout_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_2d_layout.csv'
layout = pd.read_csv(layout_path, index_col=0)
layout

,01,02,03,04,05,06,07,08,09,10,11,12
A,P102_1,P119_5,P70_5,P105_3,P59_4,P128_5,P48_4,H22,P128_1,P102_4,P102_3,P134_4
B,P70_2,P128_3,P48_4,H37,P119_1,P102_3,P105_1,P119_4,P59_1,P102_1,P128_4,P70_4
C,P48_2,P59_2,P70_4,P48_3,P105_1,P48_3,P83_2,P134_3,P59_4,H22,P134_2,P83_4
D,H26,P70_1,P134_4,P48_1,P83_1,P128_3,P128_1,H47,P119_2,P134_1,P70_2,P134_2
E,H37,P119_5,P48_2,P83_3,P102_4,P134_3,P105_2,P70_3,P102_2,P119_3,P83_4,P83_2
F,P119_2,P83_1,P59_5,P48_5,P128_5,P105_4,P128_2,P59_3,P59_2,P70_5,P105_3,H47
G,P105_2,P119_4,P48_1,P105_5,P70_3,H26,P105_5,P134_5,P83_5,P102_2,P134_5,P48_5
H,P59_3,P59_1,P70_1,P119_3,P119_1,P128_4,P105_4,P83_3,P134_1,P83_5,P59_5,P128_2


In [ ]:
files

['/data/hschlueter/pbmc5/raw/Plate7/WellB05_ChannelDAPI_Seq0018.nd2',
 '/data/hschlueter/pbmc5/raw/Plate7/WellD10_ChannelDAPI_Seq0037.nd2',
 '/data/hschlueter/pbmc5/raw/Plate7/WellD09_ChannelDAPI_Seq0038.nd2',
 '/data/hschlueter/pbmc5/raw/Plate7/WellG06_ChannelDAPI_Seq0076.nd2',
 '/data/hschlueter/pbmc5/raw/Plate7/WellE09_ChannelDAPI_Seq0055.nd2',
 '/data/hschlueter/pbmc5/raw/Plate7/WellB09_ChannelDAPI_Seq0014.nd2',
 '/data/hschlueter/pbmc5/raw/Plate7/WellH04_ChannelDAPI_Seq0091.nd2',
 '/data/hschlueter/pbmc5/raw/Plate7/WellA09_ChannelDAPI_Seq0008.nd2',
 '/data/hschlueter/pbmc5/raw/Plate7/WellH12_ChannelDAPI_Seq0083.nd2',
 '/data/hschlueter/pbmc5/raw/Plate7/WellH01_ChannelDAPI_Seq0094.nd2',
 '/data/hschlueter/pbmc5/raw/Plate7/WellE12_ChannelDAPI_Seq0058.nd2',
 '/data/hschlueter/pbmc5/raw/Plate7/WellC04_ChannelDAPI_Seq0026.nd2',
 '/data/hschlueter/pbmc5/raw/Plate7/WellA12_ChannelDAPI_Seq0011.nd2',
 '/data/hschlueter/pbmc5/raw/Plate7/WellG09_ChannelDAPI_Seq0079.nd2',
 '/data/hschlueter/p

In [ ]:
info_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_layout.csv'
info = pd.read_csv(info_path, index_col=0)
info[['patient', 'time']] = info['sample'].str.split('_', expand=True)
info['nd2'] = ''
for fname in files:
  well = fname.split('Well')[1][:3]
  info.loc[well, 'nd2'] = fname

print(info[(info['nd2'] == '') | info['patient'].isna()])  # empty wells
info = info[info['nd2'] != '']
info

             condition  sample patient time nd2
well                                           
B10   Meningioma group  P102_1    P102    1    


,condition,sample,patient,time,nd2
well,,,,,
A01,Meningioma group,P102_1,P102,1,/data/hschlueter/pbmc5/raw/Plate7/WellA01_Chan...
A02,Meningioma group,P119_5,P119,5,/data/hschlueter/pbmc5/raw/Plate7/WellA02_Chan...
A03,Meningioma group,P70_5,P70,5,/data/hschlueter/pbmc5/raw/Plate7/WellA03_Chan...
A04,Meningioma group,P105_3,P105,3,/data/hschlueter/pbmc5/raw/Plate7/WellA04_Chan...
A05,Meningioma group,P59_4,P59,4,/data/hschlueter/pbmc5/raw/Plate7/WellA05_Chan...
...,...,...,...,...,...
H08,Meningioma group,P83_3,P83,3,/data/hschlueter/pbmc5/raw/Plate7/WellH08_Chan...
H09,Meningioma group,P134_1,P134,1,/data/hschlueter/pbmc5/raw/Plate7/WellH09_Chan...
H10,Meningioma group,P83_5,P83,5,/data/hschlueter/pbmc5/raw/Plate7/WellH10_Chan...


In [ ]:
save_dir = f'/home/hschlueter/data/pbmc5/seg_voronoi_otsu/{plate}/segmentation/'
options = list(product(info.index, np.arange(11)))
for args in tqdm(options):
  segment_vornoi_otsu(args)

  0%|          | 0/1045 [00:00<?, ?it/s]

size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
s

# Plate 8

In [ ]:
plate = 'Plate8'
s_plate = 'plate_8'

In [ ]:
raw_dir = f'/data/hschlueter/pbmc5/raw/{plate}/'
files = glob.glob(f'{raw_dir}/*.nd2')
layout_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_2d_layout.csv'
layout = pd.read_csv(layout_path, index_col=0)
layout

,01,02,03,04,05,06,07,08,09,10,11,12
A,P162_5,P17_3,P65_1,P17_1,P162_4,P65_3,P37_1,P36_4,P109_1,P42_3,P42_1,P162_3
B,P36_3,H26,P36_4,P58_3,P13_2,P13_3,P109_4,P17_3,P37_3,P162_4,P27_1,H20
C,P37_3,P13_1,P13_3,P13_5,P36_5,P162_5,H47,P58_1,P162_1,H47,P27_1,P109_5
D,P32_3,P65_1,P42_2,P109_3,P162_1,P27_3,P32_1,H26,P162_3,P42_2,P36_3,P42_4
E,P27_3,P17_1,P42_4,P58_5,P58_4,P58_2,P42_5,P32_2,P13_1,P17_2,P36_1,P13_4
F,P58_2,P13_2,P36_1,P42_1,P42_3,P37_1,P162_2,P109_5,P37_2,P162_2,P58_5,P58_1
G,P58_3,P13_4,NaN,H36,NaN,P65_5,P109_1,P32_3,P27_2,H20,P65_3,P32_1
H,P109_4,P58_4,P13_5,P36_5,P37_2,P65_5,P109_3,P27_2,H36,P42_5,P17_2,P32_2


In [ ]:
files

['/data/hschlueter/pbmc5/raw/Plate8/WellE12_ChannelDAPI_Seq0050.nd2',
 '/data/hschlueter/pbmc5/raw/Plate8/WellG09_ChannelDAPI_Seq0067.nd2',
 '/data/hschlueter/pbmc5/raw/Plate8/WellE06_ChannelDAPI_Seq0045.nd2',
 '/data/hschlueter/pbmc5/raw/Plate8/WellG02_ChannelDAPI_Seq0063.nd2',
 '/data/hschlueter/pbmc5/raw/Plate8/WellF10_ChannelDAPI_Seq0053.nd2',
 '/data/hschlueter/pbmc5/raw/Plate8/WellA06_ChannelDAPI_Seq0004.nd2',
 '/data/hschlueter/pbmc5/raw/Plate8/WellF02_ChannelDAPI_Seq0060.nd2',
 '/data/hschlueter/pbmc5/raw/Plate8/WellE07_ChannelDAPI_Seq0046.nd2',
 '/data/hschlueter/pbmc5/raw/Plate8/WellF12_ChannelDAPI_Seq0051.nd2',
 '/data/hschlueter/pbmc5/raw/Plate8/WellH12_ChannelDAPI_Seq0071.nd2',
 '/data/hschlueter/pbmc5/raw/Plate8/WellG07_ChannelDAPI_Seq0066.nd2',
 '/data/hschlueter/pbmc5/raw/Plate8/WellH08_ChannelDAPI_Seq0075.nd2',
 '/data/hschlueter/pbmc5/raw/Plate8/WellB12_ChannelDAPI_Seq0009.nd2',
 '/data/hschlueter/pbmc5/raw/Plate8/WellA05_ChannelDAPI_Seq0003.nd2',
 '/data/hschlueter/p

In [ ]:
info_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_layout.csv'
info = pd.read_csv(info_path, index_col=0)
info[['patient', 'time']] = info['sample'].str.split('_', expand=True)
info['nd2'] = ''
for fname in files:
  well = fname.split('Well')[1][:3]
  info.loc[well, 'nd2'] = fname

print(info[(info['nd2'] == '') | info['patient'].isna()])  # empty wells
info = info[info['nd2'] != '']
info

             condition  sample patient time nd2
well                                           
A04   Meningioma group   P17_1     P17    1    
A08   Meningioma group   P36_4     P36    4    
A11   Meningioma group   P42_1     P42    1    
B05   Meningioma group   P13_2     P13    2    
B06   Meningioma group   P13_3     P13    3    
B08   Meningioma group   P17_3     P17    3    
C09   Meningioma group  P162_1    P162    1    
D10   Meningioma group   P42_2     P42    2    
E11   Meningioma group   P36_1     P36    1    
F03   Meningioma group   P36_1     P36    1    
G03                NaN     NaN     NaN  NaN    
G05                NaN     NaN     NaN  NaN    
G08   Meningioma group   P32_3     P32    3    


,condition,sample,patient,time,nd2
well,,,,,
A01,Meningioma group,P162_5,P162,5,/data/hschlueter/pbmc5/raw/Plate8/WellA01_Chan...
A02,Meningioma group,P17_3,P17,3,/data/hschlueter/pbmc5/raw/Plate8/WellA02_Chan...
A03,Meningioma group,P65_1,P65,1,/data/hschlueter/pbmc5/raw/Plate8/WellA03_Chan...
A05,Meningioma group,P162_4,P162,4,/data/hschlueter/pbmc5/raw/Plate8/WellA05_Chan...
A06,Meningioma group,P65_3,P65,3,/data/hschlueter/pbmc5/raw/Plate8/WellA06_Chan...
...,...,...,...,...,...
H08,Meningioma group,P27_2,P27,2,/data/hschlueter/pbmc5/raw/Plate8/WellH08_Chan...
H09,Healthy,H36,H36,None,/data/hschlueter/pbmc5/raw/Plate8/WellH09_Chan...
H10,Meningioma group,P42_5,P42,5,/data/hschlueter/pbmc5/raw/Plate8/WellH10_Chan...


In [ ]:
save_dir = f'/home/hschlueter/data/pbmc5/seg_voronoi_otsu/{plate}/segmentation/'
options = list(product(info.index, np.arange(11)))
for args in tqdm(options):
  segment_vornoi_otsu(args)

  0%|          | 0/913 [00:00<?, ?it/s]

size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
s

# Plate 9

In [ ]:
plate = 'Plate9'
s_plate = 'plate_9'

In [ ]:
raw_dir = f'/data/hschlueter/pbmc5/raw/{plate}/'
files = glob.glob(f'{raw_dir}/*.nd2')
layout_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_2d_layout.csv'
layout = pd.read_csv(layout_path, index_col=0)
layout

,01,02,03,04,05,06,07,08,09,10,11,12
A,P86_4,P39_5,P62_3,P133_3,P86_1,P133_2,H36,P157_3,P33_2,P86_2,P157_4,P38_3
B,P133_1,P133_4,P38_5,P156_2,H33,P38_4,P86_1,P62_1,P33_2,P157_4,P38_1,H20
C,P39_1,P64_1,P64_5,H36,P133_1,P39_3,P133_2,P157_3,P64_4,P38_5,P157_5,P64_3
D,P33_5,P86_3,P156_2,P64_3,H32,P157_5,P157_2,P86_3,P62_5,P39_1,P33_3,H32
E,P156_5,P133_3,P62_3,P39_4,P64_1,P64_5,P157_2,P64_2,P33_4,P86_2,P33_1,P156_1
F,H33,P64_4,P38_3,P156_4,P38_2,P39_2,P86_4,P156_1,P39_5,P133_5,P39_2,P156_3
G,P62_1,P62_4,P156_4,P157_1,P38_2,P62_2,H20,P156_3,P156_5,P33_5,P62_4,P157_1
H,P39_4,P62_5,P62_2,P133_5,P39_3,P33_4,P33_3,P133_4,P38_1,P33_1,P38_4,P64_2


In [ ]:
files

['/data/hschlueter/pbmc5/raw/Plate9/WellB05_ChannelDAPI_Seq0018.nd2',
 '/data/hschlueter/pbmc5/raw/Plate9/WellD10_ChannelDAPI_Seq0037.nd2',
 '/data/hschlueter/pbmc5/raw/Plate9/WellD09_ChannelDAPI_Seq0038.nd2',
 '/data/hschlueter/pbmc5/raw/Plate9/WellG06_ChannelDAPI_Seq0076.nd2',
 '/data/hschlueter/pbmc5/raw/Plate9/WellE09_ChannelDAPI_Seq0055.nd2',
 '/data/hschlueter/pbmc5/raw/Plate9/WellB09_ChannelDAPI_Seq0014.nd2',
 '/data/hschlueter/pbmc5/raw/Plate9/WellH04_ChannelDAPI_Seq0091.nd2',
 '/data/hschlueter/pbmc5/raw/Plate9/WellA09_ChannelDAPI_Seq0008.nd2',
 '/data/hschlueter/pbmc5/raw/Plate9/WellH12_ChannelDAPI_Seq0083.nd2',
 '/data/hschlueter/pbmc5/raw/Plate9/WellH01_ChannelDAPI_Seq0094.nd2',
 '/data/hschlueter/pbmc5/raw/Plate9/WellE12_ChannelDAPI_Seq0058.nd2',
 '/data/hschlueter/pbmc5/raw/Plate9/WellC04_ChannelDAPI_Seq0026.nd2',
 '/data/hschlueter/pbmc5/raw/Plate9/WellG09_ChannelDAPI_Seq0079.nd2',
 '/data/hschlueter/pbmc5/raw/Plate9/WellH03_ChannelDAPI_Seq0092.nd2',
 '/data/hschlueter/p

In [ ]:
info_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_layout.csv'
info = pd.read_csv(info_path, index_col=0)
info[['patient', 'time']] = info['sample'].str.split('_', expand=True)
info['nd2'] = ''
for fname in files:
  well = fname.split('Well')[1][:3]
  info.loc[well, 'nd2'] = fname

print(info[(info['nd2'] == '') | info['patient'].isna()])  # empty wells
info = info[info['nd2'] != '']
info

             condition sample patient time nd2
well                                          
A12   Meningioma group  P38_3     P38    3    


,condition,sample,patient,time,nd2
well,,,,,
A01,Meningioma group,P86_4,P86,4,/data/hschlueter/pbmc5/raw/Plate9/WellA01_Chan...
A02,Meningioma group,P39_5,P39,5,/data/hschlueter/pbmc5/raw/Plate9/WellA02_Chan...
A03,Meningioma group,P62_3,P62,3,/data/hschlueter/pbmc5/raw/Plate9/WellA03_Chan...
A04,Meningioma group,P133_3,P133,3,/data/hschlueter/pbmc5/raw/Plate9/WellA04_Chan...
A05,Meningioma group,P86_1,P86,1,/data/hschlueter/pbmc5/raw/Plate9/WellA05_Chan...
...,...,...,...,...,...
H08,Meningioma group,P133_4,P133,4,/data/hschlueter/pbmc5/raw/Plate9/WellH08_Chan...
H09,Meningioma group,P38_1,P38,1,/data/hschlueter/pbmc5/raw/Plate9/WellH09_Chan...
H10,Meningioma group,P33_1,P33,1,/data/hschlueter/pbmc5/raw/Plate9/WellH10_Chan...


In [ ]:
save_dir = f'/home/hschlueter/data/pbmc5/seg_voronoi_otsu/{plate}/segmentation/'
options = list(product(info.index, np.arange(11)))
for args in tqdm(options):
  segment_vornoi_otsu(args)

  0%|          | 0/1045 [00:00<?, ?it/s]

size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
s

# Plate 10

In [ ]:
plate = 'Plate10'
s_plate = 'plate_10'

In [ ]:
raw_dir = f'/data/hschlueter/pbmc5/raw/{plate}/'
files = glob.glob(f'{raw_dir}/*.nd2')
layout_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_2d_layout.csv'
layout = pd.read_csv(layout_path, index_col=0)
layout

,01,02,03,04,05,06,07,08,09,10,11,12
A,126_5,P99_2,P43_3,P43_2,P131_4,P126_4,P43_5,NaN,P43_4,P99_1,P99_5,P144_4
B,P138_5,P112_5,P126_2,P144_4,H16,P144_3,P131_3,P112_1,P90_3,P144_3,P131_5,P144_1
C,P112_4,H33,P90_4,P90_2,P131_1,P43_2,P99_5,P126_3,P99_3,P76_2,H32,P76_5
D,P138_2,P138_1,P90_5,P138_4,P144_2,P138_2,P90_4,126_5,P144_1,P138_1,P131_5,H29
E,H33,P43_4,P43_3,P76_5,P43_1,P90_1,P76_3,P99_1,P138_5,P99_2,P126_1,P90_2
F,P90_5,P131_4,P112_3,P90_3,H16,P131_2,P43_1,H29,P76_1,P138_3,P90_1,P131_2
G,P76_1,P112_4,P43_5,P76_3,P99_4,P112_2,P138_3,NaN,P138_4,P126_4,P112_1,P76_2
H,P144_2,P99_4,P131_1,P126_2,P112_2,P126_3,P99_3,P131_3,P112_5,H32,P126_1,P112_3


In [ ]:
files

['/data/hschlueter/pbmc5/raw/Plate10/WellC01_ChannelDAPI_Seq0020.nd2',
 '/data/hschlueter/pbmc5/raw/Plate10/WellG05_ChannelDAPI_Seq0070.nd2',
 '/data/hschlueter/pbmc5/raw/Plate10/WellE08_ChannelDAPI_Seq0050.nd2',
 '/data/hschlueter/pbmc5/raw/Plate10/WellB12_ChannelDAPI_Seq0009.nd2',
 '/data/hschlueter/pbmc5/raw/Plate10/WellC05_ChannelDAPI_Seq0024.nd2',
 '/data/hschlueter/pbmc5/raw/Plate10/WellB04_ChannelDAPI_Seq0016.nd2',
 '/data/hschlueter/pbmc5/raw/Plate10/WellH11_ChannelDAPI_Seq0078.nd2',
 '/data/hschlueter/pbmc5/raw/Plate10/WellG01_ChannelDAPI_Seq0066.nd2',
 '/data/hschlueter/pbmc5/raw/Plate10/WellG11_ChannelDAPI_Seq0075.nd2',
 '/data/hschlueter/pbmc5/raw/Plate10/WellD10_ChannelDAPI_Seq0034.nd2',
 '/data/hschlueter/pbmc5/raw/Plate10/WellH10_ChannelDAPI_Seq0079.nd2',
 '/data/hschlueter/pbmc5/raw/Plate10/WellF11_ChannelDAPI_Seq0055.nd2',
 '/data/hschlueter/pbmc5/raw/Plate10/WellG12_ChannelDAPI_Seq0076.nd2',
 '/data/hschlueter/pbmc5/raw/Plate10/WellF07_ChannelDAPI_Seq0059.nd2',
 '/dat

In [ ]:
info_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_layout.csv'
info = pd.read_csv(info_path, index_col=0)
info[['patient', 'time']] = info['sample'].str.split('_', expand=True)
info['nd2'] = ''
for fname in files:
  well = fname.split('Well')[1][:3]
  info.loc[well, 'nd2'] = fname

print(info[(info['nd2'] == '') | info['patient'].isna()])  # empty wells
info = info[info['nd2'] != '']
info

           condition sample patient time nd2
well                                        
A08              NaN    NaN     NaN  NaN    
A10   Chordoma group  P99_1     P99    1    
A11   Chordoma group  P99_5     P99    5    
B09   Chordoma group  P90_3     P90    3    
E03   Chordoma group  P43_3     P43    3    
E10   Chordoma group  P99_2     P99    2    
G08              NaN    NaN     NaN  NaN    


,condition,sample,patient,time,nd2
well,,,,,
A01,Chordoma group,126_5,126,5,/data/hschlueter/pbmc5/raw/Plate10/WellA01_Cha...
A02,Chordoma group,P99_2,P99,2,/data/hschlueter/pbmc5/raw/Plate10/WellA02_Cha...
A03,Chordoma group,P43_3,P43,3,/data/hschlueter/pbmc5/raw/Plate10/WellA03_Cha...
A04,Chordoma group,P43_2,P43,2,/data/hschlueter/pbmc5/raw/Plate10/WellA04_Cha...
A05,Chordoma group,P131_4,P131,4,/data/hschlueter/pbmc5/raw/Plate10/WellA05_Cha...
...,...,...,...,...,...
H08,Chordoma group,P131_3,P131,3,/data/hschlueter/pbmc5/raw/Plate10/WellH08_Cha...
H09,Chordoma group,P112_5,P112,5,/data/hschlueter/pbmc5/raw/Plate10/WellH09_Cha...
H10,Healthy,H32,H32,None,/data/hschlueter/pbmc5/raw/Plate10/WellH10_Cha...


In [ ]:
save_dir = f'/home/hschlueter/data/pbmc5/seg_voronoi_otsu/{plate}/segmentation/'
options = list(product(info.index, np.arange(11)))
for args in tqdm(options):
  segment_vornoi_otsu(args)

  0%|          | 0/979 [00:00<?, ?it/s]

size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
s

In [ ]:
save_dir = f'/home/hschlueter/data/pbmc5/seg_voronoi_otsu/{plate}/re_manual/segmentation/'
options = []
for well in info.index:
  stacks = nd2.imread(info.loc[well, 'nd2'])
  for i in range(stacks.shape[0]):
    options.append((well, i))

for args in tqdm(options):
  segment_vornoi_otsu(args)

  0%|          | 0/95 [00:00<?, ?it/s]

size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19


# Plate 11

In [ ]:
plate = 'Plate11'
s_plate = 'plate_11'

In [ ]:
raw_dir = f'/data/hschlueter/pbmc5/raw/{plate}/'
files = glob.glob(f'{raw_dir}/*.nd2')
layout_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_2d_layout.csv'
layout = pd.read_csv(layout_path, index_col=0)
layout

,01,02,03,04,05,06,07,08,09,10,11,12
A,P165_3,P143_4,P163_1,P163_3,P152_1,P165_1,P69_5,P143_1,P143_5,P23_5,P25_1,P143_2
B,P160_2,P143_5,P152_1,P92_3,P25_4,P163_4,P69_4,H29,P160_1,P92_2,P160_2,P163_4
C,P23_2,P163_1,P165_5,P92_1,P165_1,H43,NaN,P92_3,P25_4,P23_4,P152_2,P165_5
D,P160_5,P23_2,P25_1,P160_3,H19,P23_3,P69_3,P25_2,P160_5,P152_4,P143_2,P152_5
E,P69_2,P152_5,P69_2,P165_4,P163_2,P152_3,P23_1,P25_2,P23_5,P92_5,H29,P92_4
F,P69_3,P143_4,P165_2,H16,P69_5,P23_4,P25_5,P143_1,P69_1,P152_3,P92_1,P160_1
G,P92_2,P165_3,P143_3,P152_2,P23_1,P143_3,P92_4,P25_3,P69_1,P92_5,P163_3,P25_5
H,H43,P152_4,H16,P160_3,P165_4,P165_2,P25_3,P23_3,H19,P69_4,P163_2,NaN


In [ ]:
files

['/data/hschlueter/pbmc5/raw/Plate11/WellB05_ChannelDAPI_Seq0018.nd2',
 '/data/hschlueter/pbmc5/raw/Plate11/WellG09_ChannelDAPI_Seq0075.nd2',
 '/data/hschlueter/pbmc5/raw/Plate11/WellH10_ChannelDAPI_Seq0080.nd2',
 '/data/hschlueter/pbmc5/raw/Plate11/WellB09_ChannelDAPI_Seq0014.nd2',
 '/data/hschlueter/pbmc5/raw/Plate11/WellE08_ChannelDAPI_Seq0051.nd2',
 '/data/hschlueter/pbmc5/raw/Plate11/WellG10_ChannelDAPI_Seq0076.nd2',
 '/data/hschlueter/pbmc5/raw/Plate11/WellE04_ChannelDAPI_Seq0047.nd2',
 '/data/hschlueter/pbmc5/raw/Plate11/WellE12_ChannelDAPI_Seq0055.nd2',
 '/data/hschlueter/pbmc5/raw/Plate11/WellA09_ChannelDAPI_Seq0008.nd2',
 '/data/hschlueter/pbmc5/raw/Plate11/WellG01_ChannelDAPI_Seq0068.nd2',
 '/data/hschlueter/pbmc5/raw/Plate11/WellC04_ChannelDAPI_Seq0026.nd2',
 '/data/hschlueter/pbmc5/raw/Plate11/WellA12_ChannelDAPI_Seq0011.nd2',
 '/data/hschlueter/pbmc5/raw/Plate11/WellH06_ChannelDAPI_Seq0083.nd2',
 '/data/hschlueter/pbmc5/raw/Plate11/WellD03_ChannelDAPI_Seq0041.nd2',
 '/dat

In [ ]:
info_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_layout.csv'
info = pd.read_csv(info_path, index_col=0)
info['patient'] = info['sample']
info['time'] = 0
info['nd2'] = ''
for fname in files:
  well = fname.split('Well')[1][:3]
  info.loc[well, 'nd2'] = fname

print(info[(info['nd2'] == '') | info['patient'].isna()])  # empty wells
info = info[info['nd2'] != '']
info

           condition  sample patient  time nd2
well                                          
B10   Chordoma group   P92_2   P92_2     0    
C07              NaN     NaN     NaN     0    
C10   Chordoma group   P23_4   P23_4     0    
D09   Chordoma group  P160_5  P160_5     0    
G02   Chordoma group  P165_3  P165_3     0    
H08   Chordoma group   P23_3   P23_3     0    
H12              NaN     NaN     NaN     0    


,condition,sample,patient,time,nd2
well,,,,,
A01,Chordoma group,P165_3,P165_3,0,/data/hschlueter/pbmc5/raw/Plate11/WellA01_Cha...
A02,Chordoma group,P143_4,P143_4,0,/data/hschlueter/pbmc5/raw/Plate11/WellA02_Cha...
A03,Chordoma group,P163_1,P163_1,0,/data/hschlueter/pbmc5/raw/Plate11/WellA03_Cha...
A04,Chordoma group,P163_3,P163_3,0,/data/hschlueter/pbmc5/raw/Plate11/WellA04_Cha...
A05,Chordoma group,P152_1,P152_1,0,/data/hschlueter/pbmc5/raw/Plate11/WellA05_Cha...
...,...,...,...,...,...
H06,Chordoma group,P165_2,P165_2,0,/data/hschlueter/pbmc5/raw/Plate11/WellH06_Cha...
H07,Chordoma group,P25_3,P25_3,0,/data/hschlueter/pbmc5/raw/Plate11/WellH07_Cha...
H09,Healthy,H19,H19,0,/data/hschlueter/pbmc5/raw/Plate11/WellH09_Cha...


In [ ]:
save_dir = f'/home/hschlueter/data/pbmc5/seg_voronoi_otsu/{plate}/segmentation/'
options = list(product(info.index, np.arange(16)))
for args in tqdm(options):
  segment_vornoi_otsu(args)

  0%|          | 0/1424 [00:00<?, ?it/s]

size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
s

# Plate 12

In [ ]:
plate = 'Plate12'
s_plate = 'plate_12'

In [ ]:
raw_dir = f'/data/hschlueter/pbmc5/raw/{plate}/'
files = glob.glob(f'{raw_dir}/*.nd2')
layout_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_2d_layout.csv'
layout = pd.read_csv(layout_path, index_col=0)
layout

,01,02,03,04,05,06,07,08,09,10,11,12
A,P118_1,P21_3,P82_1,P89_4,P118_2,P95_2,P164_3,H49,P95_4,P89_3,P71_1,P89_2
B,P95_5,P164_1,P164_4,H09,P80_2,P89_4,P21_2,P95_4,P118_1,P72_2,P118_4,P80_3
C,P80_4,P155_3,P95_1,P73_5,P21_1,P21_1,P71_1,P80_4,P95_3,P82_5,P21_5,P164_2
D,P82_3,P155_2,P80_1,P164_3,P118_3,P80_3,P164_5,P164_1,P82_2,P82_2,P80_5,P89_1
E,P72_2,P164_4,P118_5,P118_4,P73_3,P73_2,H19,H49,P118_2,P82_1,P89_5,P95_3
F,P71_3,P89_1,P164_5,P89_3,P155_1,P155_3,P95_2,P155_1,P80_2,H43,H09,P118_3
G,P80_1,P82_3,P21_5,P164_2,P71_3,P95_5,P73_4,P21_3,P73_1,P82_5,P89_2,P73_5
H,P73_4,H19,P155_2,P21_2,P80_5,P89_5,P73_2,P95_1,P73_3,P73_1,H43,P118_5


In [ ]:
files

['/data/hschlueter/pbmc5/raw/Plate12/WellE01_ChannelDAPI_Seq0048.nd2',
 '/data/hschlueter/pbmc5/raw/Plate12/WellD02_ChannelDAPI_Seq0046.nd2',
 '/data/hschlueter/pbmc5/raw/Plate12/WellB01_ChannelDAPI_Seq0023.nd2',
 '/data/hschlueter/pbmc5/raw/Plate12/WellG06_ChannelDAPI_Seq0077.nd2',
 '/data/hschlueter/pbmc5/raw/Plate12/WellA09_ChannelDAPI_Seq0008.nd2',
 '/data/hschlueter/pbmc5/raw/Plate12/WellC10_ChannelDAPI_Seq0033.nd2',
 '/data/hschlueter/pbmc5/raw/Plate12/WellE09_ChannelDAPI_Seq0056.nd2',
 '/data/hschlueter/pbmc5/raw/Plate12/WellA12_ChannelDAPI_Seq0011.nd2',
 '/data/hschlueter/pbmc5/raw/Plate12/WellG05_ChannelDAPI_Seq0076.nd2',
 '/data/hschlueter/pbmc5/raw/Plate12/WellC01_ChannelDAPI_Seq0024.nd2',
 '/data/hschlueter/pbmc5/raw/Plate12/WellD08_ChannelDAPI_Seq0040.nd2',
 '/data/hschlueter/pbmc5/raw/Plate12/WellC02_ChannelDAPI_Seq0025.nd2',
 '/data/hschlueter/pbmc5/raw/Plate12/WellB03_ChannelDAPI_Seq0021.nd2',
 '/data/hschlueter/pbmc5/raw/Plate12/WellG09_ChannelDAPI_Seq0080.nd2',
 '/dat

In [ ]:
info_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_layout.csv'
info = pd.read_csv(info_path, index_col=0)
info['patient'] = info['sample']
info['time'] = 0
info['nd2'] = ''
for fname in files:
  well = fname.split('Well')[1][:3]
  info.loc[well, 'nd2'] = fname

print(info[(info['nd2'] == '') | info['patient'].isna()])  # empty wells
info = info[info['nd2'] != '']
info

Empty DataFrame
Columns: [condition, sample, patient, time, nd2]
Index: []


,condition,sample,patient,time,nd2
well,,,,,
A01,Chordoma group,P118_1,P118_1,0,/data/hschlueter/pbmc5/raw/Plate12/WellA01_Cha...
A02,Chordoma group,P21_3,P21_3,0,/data/hschlueter/pbmc5/raw/Plate12/WellA02_Cha...
A03,Chordoma group,P82_1,P82_1,0,/data/hschlueter/pbmc5/raw/Plate12/WellA03_Cha...
A04,Chordoma group,P89_4,P89_4,0,/data/hschlueter/pbmc5/raw/Plate12/WellA04_Cha...
A05,Chordoma group,P118_2,P118_2,0,/data/hschlueter/pbmc5/raw/Plate12/WellA05_Cha...
...,...,...,...,...,...
H08,Chordoma group,P95_1,P95_1,0,/data/hschlueter/pbmc5/raw/Plate12/WellH08_Cha...
H09,Chordoma group,P73_3,P73_3,0,/data/hschlueter/pbmc5/raw/Plate12/WellH09_Cha...
H10,Chordoma group,P73_1,P73_1,0,/data/hschlueter/pbmc5/raw/Plate12/WellH10_Cha...


In [ ]:
save_dir = f'/home/hschlueter/data/pbmc5/seg_voronoi_otsu/{plate}/segmentation/'
options = list(product(info.index, np.arange(16)))
for args in tqdm(options):
  segment_vornoi_otsu(args)

  0%|          | 0/1536 [00:00<?, ?it/s]

size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
s

# Plate 13

In [ ]:
plate = 'Plate13'
s_plate = 'plate_13'

In [ ]:
raw_dir = f'/data/hschlueter/pbmc5/raw/{plate}/'
files = glob.glob(f'{raw_dir}/*.nd2')
layout_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_2d_layout.csv'
layout = pd.read_csv(layout_path, index_col=0)
layout

,01,02,03,04,05,06,07,08,09,10,11,12
A,P28_1,P127_2,P75_5,P96_5,P123_5,P123_4,P75_1,P20_3,P60_5,P20_2,P75_2,P96_2
B,P60_1,P123_1,P87_5,P96_5,P28_1,P60_5,P60_3,P20_1,P28_2,P96_2,P28_4,P28_5
C,P87_1,H48,P87_3,H48,P127_4,P111_1,H25,P87_4,P123_3,P20_5,P96_4,P96_3
D,P87_2,P96_4,P96_1,P127_2,P20_3,H09,P123_2,P20_5,P127_4,P111_4,P111_3,P111_5
E,P111_3,P28_4,P28_5,P127_1,P87_2,P75_4,P60_1,P60_4,P87_3,P96_1,P127_3,P75_4
F,P60_4,P75_1,H09,P28_3,P111_2,P127_5,P20_2,P111_5,P123_4,H49,P87_5,P127_3
G,P28_3,P123_2,P87_1,P28_2,P123_5,P75_2,P75_3,P96_3,P111_4,P75_5,P60_2,P60_2
H,H49,P127_1,P75_3,P87_4,H25,P111_1,P123_3,P127_5,P123_1,P60_3,P111_2,P20_1


In [ ]:
files

['/data/hschlueter/pbmc5/raw/Plate13/WellD10_ChannelDAPI_Seq0037.nd2',
 '/data/hschlueter/pbmc5/raw/Plate13/WellD09_ChannelDAPI_Seq0038.nd2',
 '/data/hschlueter/pbmc5/raw/Plate13/WellG06_ChannelDAPI_Seq0076.nd2',
 '/data/hschlueter/pbmc5/raw/Plate13/WellE09_ChannelDAPI_Seq0055.nd2',
 '/data/hschlueter/pbmc5/raw/Plate13/WellH04_ChannelDAPI_Seq0091.nd2',
 '/data/hschlueter/pbmc5/raw/Plate13/WellA09_ChannelDAPI_Seq0008.nd2',
 '/data/hschlueter/pbmc5/raw/Plate13/WellH12_ChannelDAPI_Seq0083.nd2',
 '/data/hschlueter/pbmc5/raw/Plate13/WellH01_ChannelDAPI_Seq0094.nd2',
 '/data/hschlueter/pbmc5/raw/Plate13/WellE12_ChannelDAPI_Seq0058.nd2',
 '/data/hschlueter/pbmc5/raw/Plate13/WellC04_ChannelDAPI_Seq0026.nd2',
 '/data/hschlueter/pbmc5/raw/Plate13/WellA12_ChannelDAPI_Seq0011.nd2',
 '/data/hschlueter/pbmc5/raw/Plate13/WellG09_ChannelDAPI_Seq0079.nd2',
 '/data/hschlueter/pbmc5/raw/Plate13/WellH03_ChannelDAPI_Seq0092.nd2',
 '/data/hschlueter/pbmc5/raw/Plate13/WellE11_ChannelDAPI_Seq0057.nd2',
 '/dat

In [ ]:
info_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_layout.csv'
info = pd.read_csv(info_path, index_col=0)
info['patient'] = info['sample']
info['time'] = 0
info['nd2'] = ''
for fname in files:
  well = fname.split('Well')[1][:3]
  info.loc[well, 'nd2'] = fname

print(info[(info['nd2'] == '') | info['patient'].isna()])  # empty wells
info = info[info['nd2'] != '']
info

            condition sample patient  time nd2
well                                          
B05   CNS-other group  P28_1   P28_1     0    


,condition,sample,patient,time,nd2
well,,,,,
A01,CNS-other group,P28_1,P28_1,0,/data/hschlueter/pbmc5/raw/Plate13/WellA01_Cha...
A02,CNS-other group,P127_2,P127_2,0,/data/hschlueter/pbmc5/raw/Plate13/WellA02_Cha...
A03,CNS-other group,P75_5,P75_5,0,/data/hschlueter/pbmc5/raw/Plate13/WellA03_Cha...
A04,CNS-other group,P96_5,P96_5,0,/data/hschlueter/pbmc5/raw/Plate13/WellA04_Cha...
A05,CNS-other group,P123_5,P123_5,0,/data/hschlueter/pbmc5/raw/Plate13/WellA05_Cha...
...,...,...,...,...,...
H08,CNS-other group,P127_5,P127_5,0,/data/hschlueter/pbmc5/raw/Plate13/WellH08_Cha...
H09,CNS-other group,P123_1,P123_1,0,/data/hschlueter/pbmc5/raw/Plate13/WellH09_Cha...
H10,CNS-other group,P60_3,P60_3,0,/data/hschlueter/pbmc5/raw/Plate13/WellH10_Cha...


In [ ]:
save_dir = f'/home/hschlueter/data/pbmc5/seg_voronoi_otsu/{plate}/segmentation/'
options = list(product(info.index, np.arange(16)))
for args in tqdm(options):
  segment_vornoi_otsu(args)

  0%|          | 0/1520 [00:00<?, ?it/s]

size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
s

# Plate 14

In [ ]:
plate = 'Plate14'
s_plate = 'plate_14'

In [ ]:
raw_dir = f'/data/hschlueter/pbmc5/raw/{plate}/'
files = glob.glob(f'{raw_dir}/*.nd2')
layout_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_2d_layout.csv'
layout = pd.read_csv(layout_path, index_col=0)
layout

,01,02,03,04,05,06,07,08,09,10,11,12
A,P35_4,P66_4,P35_2,P113_3,P30_4,P167_1,P30_5,P66_3,P91_4,H25,P66_2,P53_3
B,P19_4,P19_3,P19_1,P19_2,P30_1,P30_3,P113_2,P54_4,P30_2,P19_5,H48,P53_4
C,P54_3,P91_4,P30_2,P66_4,P54_3,P35_3,P66_2,P54_1,P113_3,P91_1,P113_5,P91_3
D,P19_5,P30_5,H25,P167_3,H45,P19_2,P158_5,P53_3,P91_2,P35_1,P30_4,P19_3
E,P91_5,H48,P167_1,P53_1,P158_2,P35_1,P158_5,P30_1,P167_2,H45,P35_3,P158_2
F,P19_1,H18,P66_1,P167_3,P91_5,P19_4,P66_3,P53_1,P35_1,P113_1,P53_2,P91_1
G,P53_2,P35_2,P158_3,P54_2,P53_5,P53_4,P91_3,P113_1,P35_4,P158_1,P113_2,P167_2
H,P54_2,P113_5,P54_4,P66_1,P35_1,P158_1,P53_5,P54_1,P158_3,P30_3,H18,P91_2


In [ ]:
files

['/data/hschlueter/pbmc5/raw/Plate14/WellC05_ChannelDAPI_Seq0014.nd2',
 '/data/hschlueter/pbmc5/raw/Plate14/WellH10_ChannelDAPI_Seq0076.nd2',
 '/data/hschlueter/pbmc5/raw/Plate14/WellH09_ChannelDAPI_Seq0075.nd2',
 '/data/hschlueter/pbmc5/raw/Plate14/WellC08_ChannelDAPI_Seq0011.nd2',
 '/data/hschlueter/pbmc5/raw/Plate14/WellD04_ChannelDAPI_Seq0022.nd2',
 '/data/hschlueter/pbmc5/raw/Plate14/WellG10_ChannelDAPI_Seq0057.nd2',
 '/data/hschlueter/pbmc5/raw/Plate14/WellD10_ChannelDAPI_Seq0028.nd2',
 '/data/hschlueter/pbmc5/raw/Plate14/WellH06_ChannelDAPI_Seq0072.nd2',
 '/data/hschlueter/pbmc5/raw/Plate14/WellH12_ChannelDAPI_Seq0078.nd2',
 '/data/hschlueter/pbmc5/raw/Plate14/WellD02_ChannelDAPI_Seq0020.nd2',
 '/data/hschlueter/pbmc5/raw/Plate14/WellF10_ChannelDAPI_Seq0052.nd2',
 '/data/hschlueter/pbmc5/raw/Plate14/WellG01_ChannelDAPI_Seq0066.nd2',
 '/data/hschlueter/pbmc5/raw/Plate14/WellA09_ChannelDAPI_Seq0008.nd2',
 '/data/hschlueter/pbmc5/raw/Plate14/WellF09_ChannelDAPI_Seq0051.nd2',
 '/dat

In [ ]:
info_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_layout.csv'
info = pd.read_csv(info_path, index_col=0)
info['patient'] = info['sample']
info['time'] = 0
info['nd2'] = ''
for fname in files:
  well = fname.split('Well')[1][:3]
  info.loc[well, 'nd2'] = fname

print(info[(info['nd2'] == '') | info['patient'].isna()])  # empty wells
info = info[info['nd2'] != '']
info

            condition sample patient  time nd2
well                                          
B08   CNS-other group  P54_4   P54_4     0    


,condition,sample,patient,time,nd2
well,,,,,
A01,CNS-other group,P35_4,P35_4,0,/data/hschlueter/pbmc5/raw/Plate14/WellA01_Cha...
A02,CNS-other group,P66_4,P66_4,0,/data/hschlueter/pbmc5/raw/Plate14/WellA02_Cha...
A03,CNS-other group,P35_2,P35_2,0,/data/hschlueter/pbmc5/raw/Plate14/WellA03_Cha...
A04,CNS-other group,P113_3,P113_3,0,/data/hschlueter/pbmc5/raw/Plate14/WellA04_Cha...
A05,CNS-other group,P30_4,P30_4,0,/data/hschlueter/pbmc5/raw/Plate14/WellA05_Cha...
...,...,...,...,...,...
H08,CNS-other group,P54_1,P54_1,0,/data/hschlueter/pbmc5/raw/Plate14/WellH08_Cha...
H09,CNS-other group,P158_3,P158_3,0,/data/hschlueter/pbmc5/raw/Plate14/WellH09_Cha...
H10,CNS-other group,P30_3,P30_3,0,/data/hschlueter/pbmc5/raw/Plate14/WellH10_Cha...


In [ ]:
save_dir = f'/home/hschlueter/data/pbmc5/seg_voronoi_otsu/{plate}/segmentation/'
options = list(product(info.index, np.arange(16)))
for args in tqdm(options):
  segment_vornoi_otsu(args)

  0%|          | 0/1520 [00:00<?, ?it/s]

size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
s

# Plate 15

In [ ]:
plate = 'Plate15'
s_plate = 'plate_15'

In [ ]:
raw_dir = f'/data/hschlueter/pbmc5/raw/{plate}/'
files = glob.glob(f'{raw_dir}/*.nd2')
layout_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_2d_layout.csv'
layout = pd.read_csv(layout_path, index_col=0)
layout

,01,02,03,04,05,06,07,08,09,10,11,12
A,P26_2,P103_2,P97_3,P85_4,P85_2,P140_2,P146_4,P146_1,P121_2,P40_5,P103_1,P121_1
B,P85_3,P140_1,P34_2,P103_3,P97_5,P97_1,P40_4,P97_5,P146_1,P146_3,P26_2,P140_4
C,P26_5,P40_3,P85_4,P140_5,P140_5,P40_1,P146_4,P85_3,H13,P159_5,P159_2,P97_2
D,P121_3,P121_2,P121_3,H18,P85_1,P159_4,P159_4,P146_2,P140_4,P97_3,P159_3,P140_1
E,P26_3,P146_3,P88_1,P85_5,P159_1,P97_2,H15,P40_3,P159_2,P85_1,P26_1,P34_1
F,P146_2,P121_1,P26_5,P34_1,P97_4,P40_5,P121_5,P103_1,P140_2,P26_3,P140_3,P159_1
G,P121_5,P159_5,H45,H45,H18,P85_2,H15,P40_1,P103_2,P121_4,P40_2,P26_1
H,P140_3,P40_4,P159_3,P103_3,P40_2,P97_1,P85_5,P121_4,P97_4,H13,P34_2,P88_1


In [ ]:
files

['/data/hschlueter/pbmc5/raw/Plate15/WellB05_ChannelDAPI_Seq0018.nd2',
 '/data/hschlueter/pbmc5/raw/Plate15/WellD10_ChannelDAPI_Seq0037.nd2',
 '/data/hschlueter/pbmc5/raw/Plate15/WellD09_ChannelDAPI_Seq0038.nd2',
 '/data/hschlueter/pbmc5/raw/Plate15/WellG06_ChannelDAPI_Seq0076.nd2',
 '/data/hschlueter/pbmc5/raw/Plate15/WellE09_ChannelDAPI_Seq0055.nd2',
 '/data/hschlueter/pbmc5/raw/Plate15/WellB09_ChannelDAPI_Seq0014.nd2',
 '/data/hschlueter/pbmc5/raw/Plate15/WellA09_ChannelDAPI_Seq0008.nd2',
 '/data/hschlueter/pbmc5/raw/Plate15/WellE12_ChannelDAPI_Seq0058.nd2',
 '/data/hschlueter/pbmc5/raw/Plate15/WellC04_ChannelDAPI_Seq0026.nd2',
 '/data/hschlueter/pbmc5/raw/Plate15/WellA12_ChannelDAPI_Seq0011.nd2',
 '/data/hschlueter/pbmc5/raw/Plate15/WellG09_ChannelDAPI_Seq0079.nd2',
 '/data/hschlueter/pbmc5/raw/Plate15/WellE11_ChannelDAPI_Seq0057.nd2',
 '/data/hschlueter/pbmc5/raw/Plate15/WellC02_ChannelDAPI_Seq0024.nd2',
 '/data/hschlueter/pbmc5/raw/Plate15/WellD07_ChannelDAPI_Seq0040.nd2',
 '/dat

In [ ]:
info_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_layout.csv'
info = pd.read_csv(info_path, index_col=0)
info['patient'] = info['sample']
info['time'] = 0
info['nd2'] = ''
for fname in files:
  well = fname.split('Well')[1][:3]
  info.loc[well, 'nd2'] = fname

print(info[(info['nd2'] == '') | info['patient'].isna()])  # empty wells
info = info[info['nd2'] != '']
info

                condition sample patient  time nd2
well                                              
B11   Sarcoma-Ewing group  P26_2   P26_2     0    
G12   Sarcoma-Ewing group  P26_1   P26_1     0    


,condition,sample,patient,time,nd2
well,,,,,
A01,Sarcoma-Ewing group,P26_2,P26_2,0,/data/hschlueter/pbmc5/raw/Plate15/WellA01_Cha...
A02,Sarcoma-Ewing group,P103_2,P103_2,0,/data/hschlueter/pbmc5/raw/Plate15/WellA02_Cha...
A03,Sarcoma-Ewing group,P97_3,P97_3,0,/data/hschlueter/pbmc5/raw/Plate15/WellA03_Cha...
A04,Sarcoma-Ewing group,P85_4,P85_4,0,/data/hschlueter/pbmc5/raw/Plate15/WellA04_Cha...
A05,Sarcoma-Ewing group,P85_2,P85_2,0,/data/hschlueter/pbmc5/raw/Plate15/WellA05_Cha...
...,...,...,...,...,...
H08,Sarcoma-Ewing group,P121_4,P121_4,0,/data/hschlueter/pbmc5/raw/Plate15/WellH08_Cha...
H09,Sarcoma-Ewing group,P97_4,P97_4,0,/data/hschlueter/pbmc5/raw/Plate15/WellH09_Cha...
H10,Healthy,H13,H13,0,/data/hschlueter/pbmc5/raw/Plate15/WellH10_Cha...


In [ ]:
save_dir = f'/home/hschlueter/data/pbmc5/seg_voronoi_otsu/{plate}/segmentation/'
options = list(product(info.index, np.arange(16)))
for args in tqdm(options):
  segment_vornoi_otsu(args)

  0%|          | 0/1504 [00:00<?, ?it/s]

size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
s

# Plate 16

In [ ]:
plate = 'Plate16'
s_plate = 'plate_16'

In [ ]:
raw_dir = f'/data/hschlueter/pbmc5/raw/{plate}/'
files = glob.glob(f'{raw_dir}/*.nd2')
layout_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_2d_layout.csv'
layout = pd.read_csv(layout_path, index_col=0)
layout

,01,02,03,04,05,06,07,08,09,10,11,12
A,H21,P74_4,P45_2,P11_4,NaN,P139_4,P49_5,P117_1,P151_2,P139_5,P139_3,P61_5
B,P136_1,P11_5,P45_4,P49_3,P45_3,P117_3,NaN,P139_1,P45_1,H15,P74_4,P74_3
C,P49_1,P49_1,P151_3,P139_5,P136_2,P151_2,P11_1,P11_4,P117_4,P61_3,P61_3,P45_5
D,P117_4,P136_2,P151_5,P139_2,NaN,NaN,P139_4,P74_2,H21,P139_3,P74_5,H13
E,P74_2,NaN,P136_5,NaN,P136_1,P151_3,P49_5,P61_4,P151_4,P11_3,P136_3,P45_3
F,P139_1,P11_5,P136_5,P74_3,P61_5,P49_3,P74_1,P139_2,P61_1,P11_3,P45_5,P117_5
G,P151_1,P117_5,NaN,H24,P151_1,P45_2,P151_5,P11_2,H13,P117_1,P74_1,P151_4
H,P61_4,H24,P117_3,P61_1,P136_3,P11_2,P45_4,P11_1,P74_5,NaN,P45_1,H15


In [ ]:
files

['/data/hschlueter/pbmc5/raw/Plate16/WellD12_ChannelDAPI_Seq0034.nd2',
 '/data/hschlueter/pbmc5/raw/Plate16/WellG06_ChannelDAPI_Seq0067.nd2',
 '/data/hschlueter/pbmc5/raw/Plate16/WellG04_ChannelDAPI_Seq0065.nd2',
 '/data/hschlueter/pbmc5/raw/Plate16/WellA06_ChannelDAPI_Seq0004.nd2',
 '/data/hschlueter/pbmc5/raw/Plate16/WellB09_ChannelDAPI_Seq0014.nd2',
 '/data/hschlueter/pbmc5/raw/Plate16/WellC11_ChannelDAPI_Seq0032.nd2',
 '/data/hschlueter/pbmc5/raw/Plate16/WellE06_ChannelDAPI_Seq0046.nd2',
 '/data/hschlueter/pbmc5/raw/Plate16/WellC03_ChannelDAPI_Seq0024.nd2',
 '/data/hschlueter/pbmc5/raw/Plate16/WellE08_ChannelDAPI_Seq0048.nd2',
 '/data/hschlueter/pbmc5/raw/Plate16/WellD08_ChannelDAPI_Seq0038.nd2',
 '/data/hschlueter/pbmc5/raw/Plate16/WellC01_ChannelDAPI_Seq0022.nd2',
 '/data/hschlueter/pbmc5/raw/Plate16/WellD03_ChannelDAPI_Seq0041.nd2',
 '/data/hschlueter/pbmc5/raw/Plate16/WellD04_ChannelDAPI_Seq0040.nd2',
 '/data/hschlueter/pbmc5/raw/Plate16/WellF05_ChannelDAPI_Seq0058.nd2',
 '/dat

In [ ]:
info_path = f'/data/hschlueter/pbmc5/layout/{s_plate}_layout.csv'
info = pd.read_csv(info_path, index_col=0)
info['patient'] = info['sample']
info['time'] = 0
info['nd2'] = ''
for fname in files:
  well = fname.split('Well')[1][:3]
  info.loc[well, 'nd2'] = fname

print(info[(info['nd2'] == '') | info['patient'].isna()])  # empty wells
info = info[info['nd2'] != '']
info

          condition  sample patient  time  \
well                                        
A05             NaN     NaN     NaN     0   
B07             NaN     NaN     NaN     0   
D05             NaN     NaN     NaN     0   
D06             NaN     NaN     NaN     0   
E02             NaN     NaN     NaN     0   
E04             NaN     NaN     NaN     0   
E05   Sarcoma group  P136_1  P136_1     0   
E11   Sarcoma group  P136_3  P136_3     0   
F10   Sarcoma group   P11_3   P11_3     0   
G03             NaN     NaN     NaN     0   
G08   Sarcoma group   P11_2   P11_2     0   
H10             NaN     NaN     NaN     0   

                                                    nd2  
well                                                     
A05                                                      
B07                                                      
D05                                                      
D06                                                      
E02                  

,condition,sample,patient,time,nd2
well,,,,,
A01,Healthy,H21,H21,0,/data/hschlueter/pbmc5/raw/Plate16/WellA01_Cha...
A02,Sarcoma group,P74_4,P74_4,0,/data/hschlueter/pbmc5/raw/Plate16/WellA02_Cha...
A03,Sarcoma group,P45_2,P45_2,0,/data/hschlueter/pbmc5/raw/Plate16/WellA03_Cha...
A04,Sarcoma group,P11_4,P11_4,0,/data/hschlueter/pbmc5/raw/Plate16/WellA04_Cha...
A06,Sarcoma group,P139_4,P139_4,0,/data/hschlueter/pbmc5/raw/Plate16/WellA06_Cha...
...,...,...,...,...,...
H08,Sarcoma group,P11_1,P11_1,0,/data/hschlueter/pbmc5/raw/Plate16/WellH08_Cha...
H09,Sarcoma group,P74_5,P74_5,0,/data/hschlueter/pbmc5/raw/Plate16/WellH09_Cha...
H10,NaN,NaN,NaN,0,/data/hschlueter/pbmc5/raw/Plate16/WellH10_Cha...


In [ ]:
save_dir = f'/home/hschlueter/data/pbmc5/seg_voronoi_otsu/{plate}/segmentation/'
options = list(product(info.index, np.arange(16)))
for args in tqdm(options):
  segment_vornoi_otsu(args)

  0%|          | 0/1360 [00:00<?, ?it/s]

size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
size: 19
s